In [ ]:
using Tetris
include("../core/TetrisAICore.jl")
import .TetrisAICore: loadmodel, predict, Model,
 GameState, MoveState, Action, valid_movement, 
 move, put_mino!, generate_minopos, get_node_list,
  Node, draw_game, init_screen, get_key_state, mysleep,
mino_to_array, AILux, create_model, select_node, vector2array
using CUDA
using JLD2
using HTTP
using CodecZstd
using Serialization
CUDA.math_mode!(CUDA.PEDANTIC_MATH)
module TetrisAICore
module AILux
using CUDA
using Lux, LuxCUDA, NNlib, MLUtils, Zygote
import Lux: gpu_device, cpu_device
gpu = gpu_device()
cpu = cpu_device()
export gpu, cpu
using JLD2, Optimisers
using Statistics, Random
using NamedTupleTools
include("play/layers.jl")
include("play/network.jl")
export QNetwork
end
using .AILux
export QNetwork, gpu, cpu
end
using .TetrisAICore

In [ ]:
model, ps, st = loadmodel("play/mainmodel.jld2")

In [ ]:
model

In [ ]:
NEXT位置による性能変化

In [ ]:
# 24x10
board = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 1 1 0 0 0 0 0 0
    1 1 1 1 1 0 0 0 0 0
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 1 0 
    1 1 1 1 1 1 1 1 1 0 
    1 1 1 1 1 1 1 1 1 0    
]
nothing

In [ ]:
# 24x10
minopos = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
]
nothing

In [ ]:
for i in 2:7, j in 2:7, k in 2:7, l in 2:7, m in 2:7
    length(Set([i, j, k, l, m])) != 5 && continue
    combo = 0
    btb = 0
    tspin = 0
    # [i_mino, o_mino, s_mino, z_mino, j_mino, l_mino, t_mino]
    holdnext = zeros(7, 6)
    holdnext[j, 1] = 1
    holdnext[k, 2] = 1
    holdnext[l, 3] = 1
    holdnext[m, 4] = 1
    holdnext[i, 5] = 1
    holdnext[1, 6] = 1
    board_array = reshape(board, (24, 10, 1, 1))
    minopos_array = reshape(minopos, (24, 10, 1, 1))
    combo_array = reshape([combo], (1, 1))
    btb_array = reshape([btb], (1, 1))
    tspin_array = reshape([tspin], (1, 1))
    holdnext_array = reshape(holdnext, (7, 6, 1))
    r, _ = model((board_array, minopos_array, combo_array, btb_array, tspin_array, holdnext_array), ps, st)
    println(r[1])
end
nothing

組み合わせの調査

In [ ]:
# 24x10
board = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 1 1 0 0 0 0 0 0
    1 1 1 1 1 0 0 0 0 0
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 1 0 
    1 1 1 1 1 1 1 1 1 0 
    1 1 1 1 1 1 1 1 1 0    
]
nothing

In [ ]:
# 24x10
board = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 1 1 0 0 0 0 0 0
    1 1 1 1 1 0 0 0 0 0
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 1 0 
    1 1 1 1 1 1 1 1 1 0 
    1 1 1 1 1 1 1 1 1 0    
]
nothing

In [ ]:
# 24x10
minopos = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
]
nothing

In [ ]:
target = 1
hold = 1
for i in 1:7, j in 1:7, k in 1:7, l in 1:7
    length(Set([i, j, k, l])) != 4 && continue
    target ==i || target == j || target == k || target == l && continue
    combo = 0
    btb = 0
    tspin = 0
    # [i_mino, o_mino, s_mino, z_mino, j_mino, l_mino, t_mino]
    holdnext = zeros(7, 6)
    holdnext[hold, 1] = 1
    holdnext[i, 2] = 1
    holdnext[j, 3] = 1
    holdnext[k, 4] = 1
    holdnext[l, 5] = 1
    holdnext[target, 6] = 1
    board_array = reshape(board, (24, 10, 1, 1))
    minopos_array = reshape(minopos, (24, 10, 1, 1))
    combo_array = reshape([combo], (1, 1))
    btb_array = reshape([btb], (1, 1))
    tspin_array = reshape([tspin], (1, 1))
    holdnext_array = reshape(holdnext, (7, 6, 1))
    r, _ = model((board_array, minopos_array, combo_array, btb_array, tspin_array, holdnext_array), ps, st)
    println(r[1])
end
nothing